<a href="https://colab.research.google.com/github/JupyterJones/AI-art/blob/master/Copy_of_Thin_Plate_Spline_Motion_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/yoyo-nb/Thin-Plate-Spline-Motion-Model.git

In [ ]:
cd Thin-Plate-Spline-Motion-Model

In [ ]:
#!mkdir checkpoints
!wget -c https://cloud.tsinghua.edu.cn/f/da8d61d012014b12a9e4/?dl=1 -O checkpoints/vox.pth.tar
!wget -c https://cloud.tsinghua.edu.cn/f/483ef53650b14ac7ae70/?dl=1 -O checkpoints/ted.pth.tar
!wget -c https://cloud.tsinghua.edu.cn/f/9ec01fa4aaef423c8c02/?dl=1 -O checkpoints/taichi.pth.tar
!wget -c https://cloud.tsinghua.edu.cn/f/cd411b334a2e49cdb1e2/?dl=1 -O checkpoints/mgif.pth.tar

In [ ]:
import os
os.chdir("Thin-Plate-Spline-Motion-Model")

In [ ]:
!mv ../checkpoints/*.* checkpoints 

In [ ]:
!ls checkpoints

In [7]:
import torch

# edit the config
device = torch.device('cpu')
dataset_name = 'vox' # ['vox', 'taichi', 'ted', 'mgif']
source_image_path = 'Thin-Plate-Spline-Motion-Model/assets/source.png'
driving_video_path = 'Thin-Plate-Spline-Motion-Model/assets/driving.mp4'
output_video_path = 'Thin-Plate-Spline-Motion-Model/assets/generated.mp4'
config_path = 'config/vox-256.yaml'
checkpoint_path = 'checkpoints/vox.pth.tar'
predict_mode = 'relative' # ['standard', 'relative', 'avd']
find_best_frame = True # when use the relative mode to animate a face, use 'find_best_frame=True' can get better quality result

pixel = 256 # for vox, taichi and mgif, the resolution is 256*256
if(dataset_name == 'ted'): # for ted, the resolution is 384*384
    pixel = 384

if find_best_frame:
  !pip install face_alignment

In [6]:
!ls Thin-Plate-Spline-Motion-Model/assets

 driving-.mp4   driving-old.mp4  'source (copy 1).png'	 source.png   vox.gif
 driving.mp4    generated.mp4	  source.jpg		 ted.gif


In [ ]:
try:
  import imageio
  import imageio_ffmpeg
except:
  !pip install imageio_ffmpeg
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
import os

warnings.filterwarnings("ignore")
driving_video_path="Thin-Plate-Spline-Motion-Model/assets/driving.mp4"
source_image_path="Thin-Plate-Spline-Motion-Model/assets/source.png"
source_image = imageio.imread(source_image_path)
reader = imageio.get_reader(driving_video_path)

source_image = resize(source_image, (pixel, pixel))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (pixel, pixel))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [ ]:
!ls 

In [ ]:
!cp demo.py demos.py

In [ ]:
from demos import load_checkpoints
inpainting, kp_detector, dense_motion_network, avd_network = load_checkpoints(config_path = config_path, checkpoint_path = checkpoint_path, device = device)

In [ ]:
from demos import make_animation
from skimage import img_as_ubyte

if predict_mode=='relative' and find_best_frame:
    from demo import find_best_frame as _find
    i = _find(source_image, driving_video, device.type=='cpu')
    print ("Best frame: " + str(i))
    driving_forward = driving_video[i:]
    driving_backward = driving_video[:(i+1)][::-1]
    predictions_forward = make_animation(source_image, driving_forward, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)
    predictions_backward = make_animation(source_image, driving_backward, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)
    predictions = predictions_backward[::-1] + predictions_forward[1:]
else:
    predictions = make_animation(source_image, driving_video, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)

#save resulting video
imageio.mimsave(output_video_path, [img_as_ubyte(frame) for frame in predictions], fps=fps)

HTML(display(source_image, driving_video, predictions).to_html5_video())

In [ ]:
# Save resulting video
output_video_path = "animated-Image.mp4"
imageio.mimsave(output_video_path, [img_as_ubyte(frame) for frame in predictions], fps=fps)


In [ ]:
output_video_path = "../animated-Image.mp4"
imageio.mimsave(output_video_path, [img_as_ubyte(frame) for frame in predictions], fps=fps)

In [ ]:
from IPython.display import HTML

video_path = "animated-Image.mp4"

# Generate HTML code to display the video
html_code = f'<video controls><source src="{video_path}" type="video/mp4"></video>'

# Display the video in Jupyter Notebook
HTML(html_code)


In [ ]:
!ls Thin-Plate-Spline-Motion-Model/temp

In [ ]:
from moviepy.editor import VideoFileClip

# Input video file
input_file = "Thin-Plate-Spline-Motion-Model/temp/driver_orig.mp4"

# Output video file
output_file = "Thin-Plate-Spline-Motion-Model/temp/driver4.mp4"

# Crop dimensions
crop_width = 480
crop_height = 480

# Load the video clip
clip = VideoFileClip(input_file)

# Get the center coordinates for cropping
center_x = clip.size[0] // 2
center_y = clip.size[1] // 2

# Calculate the top-left corner coordinates for cropping
top_left_x = center_x - crop_width // 2
top_left_y = center_y - crop_height // 2

# Crop the video clip
cropped_clip = clip.crop(x1=top_left_x, y1=top_left_y, x2=top_left_x + crop_width, y2=top_left_y + crop_height)

# Set the audio of the cropped clip to the original audio
cropped_clip = cropped_clip.set_audio(clip.audio)

# Write the cropped clip to the output file
cropped_clip.write_videofile(output_file, codec="libx264", audio_codec="aac")

# Close the clips
clip.close()
cropped_clip.close()


In [ ]:
from moviepy.editor import VideoFileClip
import moviepy.video.fx.all as vfx

# Input video file
input_file = "Thin-Plate-Spline-Motion-Model/temp/driver_orig.mp4"

# Output video file
output_file = "Thin-Plate-Spline-Motion-Model/temp/driver2.mp4"

# Crop dimensions
crop_size = 480

# Load the video clip
clip = VideoFileClip(input_file)

# Calculate the crop dimensions
crop_x = (clip.size[0] - crop_size) // 2
crop_y = (clip.size[1] - crop_size) // 2

# Crop the video clip
cropped_clip = clip.crop(x1=crop_x, y1=crop_y, x2=crop_x + crop_size, y2=crop_y + crop_size)

# Resize the cropped clip to a square
resized_clip = cropped_clip.fx(vfx.resize, width=crop_size, height=crop_size)

# Set the audio of the resized clip to the original audio
resized_clip = resized_clip.set_audio(clip.audio)

# Write the resized clip to the output file
resized_clip.write_videofile(output_file, codec="libx264", audio_codec="aac")

# Close the clips
clip.close()
cropped_clip.close()
resized_clip.close()


In [ ]:
#input_file = "Thin-Plate-Spline-Motion-Model/temp/driver_orig.mp4"

# Output video file
#output_file = "Thin-Plate-Spline-Motion-Model/temp/driver.mp4"
!ffmpeg -i Thin-Plate-Spline-Motion-Model/temp/driver_orig.mp4 -filter:v "crop=512:512:(iw-512)/2:(ih-512)/2" \
-c:a copy Thin-Plate-Spline-Motion-Model/temp/driver.mp4


In [ ]:
import cv2
from moviepy.editor import VideoClip

def capture_frame(t):
    # Open the camera
    cap = cv2.VideoCapture(0)
    
    # Read a frame from the camera
    ret, frame = cap.read()
    
    # Close the camera
    cap.release()
    
    # Convert the frame from BGR to RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    return frame

# Create a video clip from the capture_frame function
video_clip = VideoClip(capture_frame, duration=10)

# Display the video clip
video_clip.preview()


In [ ]:
input_file = "Thin-Plate-Spline-Motion-Model/temp/drive.mp4

In [ ]:
!ffmpeg -i Thin-Plate-Spline-Motion-Model/temp/driver.mp4 -ss 00:00:05 -vframes 1 -y Thin-Plate-Spline-Motion-Model/temp/source.png


In [ ]:
!ls Thin-Plate-Spline-Motion-Model/temp